# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2018 &ndash; Week 3 &ndash; ETH Zurich</center>

## Introduction
This week we will cover mostly theoretical aspects of Hadoop and HDFS and we will discuss advantages and limitations of different storage models.
Here are some recommended references we encourage you to read:

[1] [Shvachko, K. et al. (2010). The Hadoop Distributed File System. In MSST.](https://doi.org/10.1109/MSST.2010.5496972)

[2] [White, T. (2015). Hadoop: The Definitive Guide (4th ed.). O’Reilly Media, Inc.](http://www.library.ethz.ch/DADS:default_scope:ebi01_prod010445248)

[3] [Ghemawat, S. et al. (2003). The Google File System. In SOSP (Vol. 37, p. 29)](https://doi.org/10.1145/945445.945450)

#### What is Hadoop?
*"Hadoop provides a **distributed file system** and a
**framework for the analysis and transformation** of very **large**
data sets using the MapReduce paradigm."* [1] 

Several components are part of this framework. In this course you will study HDFS, MapReduce and HBase while this exercise focuses on HDFS and storage models.


| *Component*                |*Description*  |*First developer*  |
|----------------------------------------------|---|---|
| **HDFS**                  |Distributed file system  |Yahoo!  |
| **MapReduce**   |Distributed computation framework   |Yahoo!  |
| **HBase**           | Column-oriented table service  |Powerset (Microsoft)  |
| Pig  | Dataflow language and parallel execution framework  |Yahoo!   |
| Hive            |Data warehouse infrastructure   |Facebook  |
| ZooKeeper    |Distributed coordination service   |Yahoo!  |
| Chukwa  |System for collecting management data   |Yahoo!  |
| Avro                |Data serialization system   |Yahoo! + Cloudera  |

## 1. The Hadoop Distributed File System
### 1.1 &ndash; State which of the following statements are true:

1. The HDFS namespace is a hierarchy of files and directories.

1. In HDFS, each block of the file is either 64 or 128 megabytes depending on the version and distribution of Hadoop in use, and this cannot be changed.

1. A client wanting to write a file into HDFS, first contacts the NameNode, then sends the data to it. The NameNode will write the data into multiple DataNodes in a pipelined fashion. 

1. A DataNode may execute multiple application tasks for different clients concurrently.

1. The cluster can have thousands of DataNodes and tens of thousands of HDFS clients per cluster.

1. HDFS NameNodes keep the namespace in RAM.

1. The locations of block replicas are part of the persistent checkpoint that the NameNode stores in its native file system.

1. If the block size is set to 64 megabytes, storing a file of 80 megabytes will actually require 128 megabytes of physical memory (2 blocks of 64 megabytes each). 


**Solution**

1. True, in contrast with the Object Storage logic model, HDFS is designed to handle a relatively small amount of huge files. A hierarchical file system can therefore be handled efficiently by a single NameNode.

1. False, the default size is either 64 or 128 megabytes but this can be easily changed in the configuration.

1. False, the client writes data to the DataNodes. No data goes through the NameNode.

1. True, each DataNode may execute multiple application tasks concurrently.

1. True, since each DataNode can execute multiple tasks concurrently, there may be more clients than DataNodes.

1. True, and an image of such namespace is also persisted in the NameNode file system.

1. False, the locations of block replicas may change over time and are not part of the persistent checkpoint.

1. False, the size of the data file equals the actual length of the block and does not require extra space to round it up to the nominal block size as in traditional file systems. Therefore 80 megabytes will be stored as a block of 64 megabytes + a block of 16 megabytes.

### 1.2 &ndash; A typical filesystem block size is 4096 bytes. How large is a block in HDFS? List at least two advantages of such choice.

** Solution **

Typical size for a block is either 64 or 128 megabytes. A large chunk size offers several important advantages. 

0. it minimizes the cost of seeks. If the block is large enough, the time it takes to transfer the data from the disk can be significantly longer than the time to seek to the start of the block. Thus, transferring a large file made of multiple blocks operates at the disk transfer rate.

1. it reduces clients' need to interact with the master because reads and writes on the same chunk require only one initial request to the master for chunk location information. The reduction is especially significant for our workloads because applications mostly read and write large files sequentially. 

2. since on a large chunk, a client is more likely to perform many operations on a given chunk, it can reduce network overhead by keeping a persistent TCP connection to the chunkserver over an extended period of time. 

3. it reduces the size of the metadata stored on the master. This allows us to keep the metadata in memory.

### 1.3 &ndash; How does the hardware cost grow as function of the amount of data we need to store in a Distributed File System such as HDFS? Why?


** Solution **

**Linearly**. HDFS is designed taking machine failure into account, and therefore DataNodes do not need to be (highly expensive) highly reliable machines. Instead standard commodity hardware can be used. Moreover the number of nodes can be increased as soon as it becomes necessary, avoiding wasting of resources when the amount of data is still limited. This is indeed the main advantage of scaling out compared to scaling up, which has exponential cost growth.


### 1.4 &ndash; Single Point of Failure

1. Which component is the main single point of failure in Hadoop?

1. What is the Secondary NameNode?

** Solution **


1. Prior to Hadoop 2.0.0, the **NameNode was a single point of failure**. While the loss of any other machine (intermittently or permanently) does not result in data loss, NameNode loss results in cluster unavailability. The permanent loss of NameNode data would render the cluster's HDFS inoperable.
The HDFS High Availability feature addresses the above problems by providing the option of running two redundant NameNodes in the same cluster in an Active/Passive configuration with a hot standby. 

1. The Secondary NameNode is a node that merges the fsimage and the edits log files periodically and keeps edits log size within a limit. This allows the NameNode to start up faster in case of failure, but the Secondary NameNode is not a redundant NameNode.

### 1.5 &ndash; Scalability, Durability and Performance on HDFS
Explain how HDFS accomplishes the following requirements:

1. Scalability

1. Durability

1. High sequential read/write performance

** Solution **

1. Scalability: by partitioning files into blocks and distributing them to many servers operating in parallel, HDFS can scale to potentially unlimited number of files of any size. By adding more DataNodes the storage capacity of the system can be increased arbitrarily. It has been demonstrated to scale beyond tens of petabytes (PB). More importantly, it does so with linear performance characteristics and cost.

1. Durability: HDFS creates multiple copies of each block (by default 3, on different racks) to minimize the probability of data loss.

1. High sequential read/write performance: by splitting huge files into blocks and spreading these into multiple machines. This makes parallel reads possible (accessing different nodes at the same time) either by using multiple clients or by using a distributed data processing framework such as MapReduce.

## 2. File I/O operations and replica management.


### 2.1 &ndash; Replication policy
Assume your HDFS cluster is made of 3 racks, each containing 3 DataNodes. Assume also the HDFS is configured to use a block size of 100 megabytes and that a client is connecting from outside the datacenter (therefore no DataNode is priviledged). 

1. The client uploads a file of 150 megabytes. Draw in the picture below a possible blocks configuration according to the default HDFS replica policy. How many replicas are there for each block? Where are these replicas stored?

1. Can you find a with a different policy that, using the same number of replicas, improves the expected availability of a block? Does your solution show any drawbacks?

1. Referring to the picture below, assume a block is stored in Node 3, as well as in Node 4 and Node 5. If this block of data has to be processed by a task running on Node 6, which of the three replicas will be actually read by Node 6? 


![Cluster](https://ethbigdata2017.blob.core.windows.net/exercise3/cluster.jpg)

** Solution **

1. For each block independently, the HDFS's placement policy is to put one replica on a random node in a random rack, another on one node in a different rack, and the last on a different node in the same rack chosen for the second replica. A possibile configuration is shown in the picture (but there are many more valid solutions).

1. One could decide to store the 3 replicas in 3 different racks, increasing the expected availability. However this would also slow down the writing process that would involve two inter-rack communications instead of one. Usually, the probability of failure of an entire rack is much smaller than the probability of failure of a node and therefore it is a good tradeoff to have 2/3 of the replicas in one rack.

1. Either the one stored in Node 4 or Node 5, assuming the intra-rack topology is such that the distance from these nodes to Node 6 is the same. In general, the reading priority is only based on the distance, not on which node was first selected in the writing process.

![Cluster](https://ethbigdata2017.blob.core.windows.net/exercise3/cluster_solut.jpg)

### 2.2 &ndash; File read and write data flow.
To get an idea of how data flows between the client interacting with HDFS, consider a diagram below which shows main components of HDFS. 

![clientHDFS](https://ethbigdata2017.blob.core.windows.net/exercise3/clientHDFS.jpg)

1. Draw the main sequence of events when a client copies a file to HDFS.
2. Draw the main sequence of events when a client reads a file from HDFS.
3. Why do you think a client writes data directly to datanodes instead of sending it through the namenode?

** Solution **

1. Steps 1-4 are applied for each block of the file. <br>
   1 - HDFS client asks the Namenode for a DataNode to host replica of the i-th block of the file. <br>
   2 - NameNode replies with a location of the DataNode for i-th block <br>
   3 - The client writes i-th block to DataNodes in pipeline fashion. <br>
   4 - DataNodes in the write pipeline acknowledge the writing of a block. Once all of them replied, the first contacted DataNode replies with acknowledgement to the client. <br>
   5 - the client sends to the NameNode a request to close the file 

![clientHDFS](https://ethbigdata2017.blob.core.windows.net/exercise3/HDFSClientWriteFlow.png)

2. <br>
   1 - HDFS client request a file <br>
   2 - NameNode replies with a list of blocks and the locations of each replica. <br>
   3 - The client reads each block from the closest datanode.

![clientHDFS](https://ethbigdata2017.blob.core.windows.net/exercise3/clientHDFSsolWrite.jpg)

3. if the namenode was responsible for copying all files to datanodes. then it would become a bottleneck.

### 2.3 &ndash; Network topology.
HDFS estimates the network bandwidth between two nodes by their distance. The distance from a node to its parent node is assumed to be one. A distance between two nodes can be calculated by summing up thier distances to their closest common ancestor. A shorter distance between two nodes means that the greater bandwidth they can utilize to transfer data. Consider a diagram of a possible hadoop cluster over two datacenters below. 

![networkHDFS](https://ethbigdata2017.blob.core.windows.net/exercise3/networkHDFS.jpg)

Calculate following distances using the distance rule explained above:
1. Node 0 and Node 1
2. Node 0 and Node 2
3. Node 1 and Node 4
4. Node 4 and Node 5
5. Node 2 and Node 3
6. Two processes of Node 1


** Solution **

1. 2
2. 4 
3. 6
4. 4
5. 2
6. 0

## 3. Storage models
### 3.1 &ndash; List two differences between Object Storage and Block Storage.

**Solution**

1. Block Storage implements file storage API, whereas Object Storage provides only key-value interface.

2. Pure Object Storage has a limit on object size, since the object cannot be partitioned across machines. Block Storage does not have this limitation and can split objects into blocks. Therefore, Block Storage can store PB files, whereas Object Storage is limited by the storage capacity of a single node. On the other hand, object storage can store more files than Block Storage.  

### 3.2 &ndash; Compare Object Storage and Block Storage. For each of the following use cases, say which technology better fits the requirements.

1. Store Netflix movie files in such a way they are accessible from many client applications at the same time *[Object storage | Block Storage ]*

1. Store experimental and simulation data from CERN *[Object storage | Block Storage ]*

1. Store the auto-backups of iPhone/Android devices *[Object storage | Block Storage ]*


**Solution**

1. Object Storage. Client applications do not need block-level access to stream the data, and an high level interface is best suited to this purpose.

1. Block Storage. Because it can handle large files and store more data than ordinary object storage.

1. Object Storage. Backups are usually written once and rarely read. When data is read, partial access to each file is not essential. The client devices do not need to know the block composition of the object being stored. In fact, Apple [publicly confirmed](http://readwrite.com/2014/08/26/apple-icloud-amazon-web-services-hosting/) that backups data for iOS is stored on Amazon S3 and Microsoft Azure.

## 4. Install Hadoop
You can now install Hadoop on your local computer to test HDFS in practice. The installation on GNU/Linux is rather easy, [follow this steps](https://hadoop.apache.org/docs/r2.7.4/hadoop-project-dist/hadoop-common/SingleCluster.html). With a similar procedure you can also install on [Windows](http://wiki.apache.org/hadoop/Hadoop2OnWindows) or [Mac OSX](http://zhongyaonan.com/hadoop-tutorial/setting-up-hadoop-2-6-on-mac-osx-yosemite.html).

During next recitation sessions, we will use a cluster version of HDFS on Azure. The purpose of this session is to familiarise you with shell commands and web Interface by using your local machines. 

Each Hadoop cluster is set up in one of the three supported modes:

- Local (Standalone) Mode
- Pseudo-Distributed Mode
- Fully-Distributed Mode

By default Hadoop runs in Local Mode but you should set-up it for **Pseudo-Distributed Mode**. This will allow you to run Hadoop on a single-node (your computer) simulating a distributed file system. As explained in the tutorials, to set up a Pseudo-Distributed Mode you will need to edit `etc/hadoop/core-site.xml` and `etc/hadoop/hdfs-site.xml` as follows:

* `etc/hadoop/core-site.xml`:

```xml
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://localhost:9000</value>
    </property>
</configuration>
```

* `etc/hadoop/hdfs-site.xml`:

```xml
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>
```

Hadoop distribution requires having preinstalled Java and knowing the root directory of it. 
A possible root directory of your already installed Java installation on Linux is 
```
 /usr/lib/jvm/default-java
```

Once you have formatted your filesystem (`$ bin/hdfs namenode -format`) and started the NameNode daemon (`$ sbin/start-dfs.sh`) you should be able to browse `http://localhost:50070/` and visualize the web interface of the daemon which should look similar to the following:

![Hadoop web interface](https://ethbigdata2017.blob.core.windows.net/exercise3/hadoop.png)

In the `Datanodes` tab you should see a single operating datanode.

### 4.1 &ndash; Upload a file into HDFS

Pick an image file in your computer (or you can also download a random one) and try to upload it to HDFS. You may need to create an empty directory before uploading.

1. Which command do you use to upload from the local file system to HDFS?

1. Which information can you find if you use `Utilities -> Browse the file system` in the daemon web interface?


** Solution**

1. It can be `
```
/bin/hadoop fs -mkdir /myfolder
/bin/hadoop fs -put ~/Documents/image.JPG /myfolder
```
1. 

|Permission |	Owner |	Group |	Size |	Last Modified |	Replication |	Block Size	| Name|
|-----------|---------|-------|------|----------------|-------------|---------------|-----|
|-rw-r--r--	|anconam|	supergroup|	3.61 MB|	10/6/2016, 11:25:27 AM	|1|	128 MB	|image.JPG|


### 4.2 &ndash; Local File System

Now take a look at the documented default values of `hdfs-default.xml` and `core-default.xml` and locate the file that you have just uploaded into HDFS.

- http://hadoop.apache.org/docs/r2.7.4/hadoop-project-dist/hadoop-hdfs/hdfs-default.xml
- http://hadoop.apache.org/docs/r2.7.4/hadoop-project-dist/hadoop-common/core-default.xml


1. Is the image still accessible with a normal image viewer? 

1. What is the path of the image relative to the HDFS filesystem?

1. What is the path of the image relative to your OS filesystem?

** Solution**

1. Yes, it is still visible and readable.
1. In my case it is `myfolder/image.JPG`.
1. In my case it is `/tmp/hadoop-anconam/dfs/data/current/BP-92054669-127.0.1.1-1475745494816/current/finalized/subdir0/subdir0/blk_1073741825`.

### 4.3 &ndash; Local File System

Now try to upload a file that is >150MB. On Unix-based system you can also generate such a file filled with zero using:

```
$ dd if=/dev/zero of=zeros.dat bs=1M count=150
```

1. How many blocks the file is split into?

**Solution**
1. Two, one of 134217728 bytes and the other of 23068672 bytes.